# Convirtiendo datos a ttl
:D


In [ ]:
import pandas as pd # no quitar, se rompe el proyectoimport pandas as pd

In [ ]:
#Leemos TODO PORQUE SI
circuits = pd.read_csv('data/circuits.csv')
constructorResults = pd.read_csv('data/constructor_results.csv')
constructors = pd.read_csv('data/constructors.csv')
constructorStandings = pd.read_csv('data/constructor_standings.csv')
drivers = pd.read_csv('data/drivers.csv')
driverStandings = pd.read_csv('data/driver_standings.csv')
lapTimes = pd.read_csv('data/lap_times.csv')
pitStops = pd.read_csv('data/pit_stops.csv')
qualifying = pd.read_csv('data/qualifying.csv')

In [ ]:
#Escribimos en el archivo data lo primero de la salida
with open('data.txt', 'w') as file:
    #Vaciamos el archivo
    file.write("@prefix ex: <http://ex.org/>.\n @prefix owl: <http://www.w3.org/2002/07/owl#>.\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.")
    
"""
PARA EL RESTO DEBEN ESCRIBIR CON
with open('data.txt', 'a') as file:
    file.write("LO QUE SEA")
"""


# Generar RDF Nationality

In [ ]:
# TODO: HACER LA WEA

# Generar RDF Pistas

In [ ]:
data_rdf = ""

#Circuits


# Generar RDF Constructores